In [10]:
import os
import math

import time
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import cm as cm
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.covariance import EllipticEnvelope
from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
from xgboost.sklearn import XGBClassifier

import pickle

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline

In [11]:
_train_=pd.read_csv('../input/train.csv')
places=_train_['place_id'].unique()

_train_.sort(['time'], inplace=True)
trainRatio=0.7
train=_train_[:int(len(_train_)*trainRatio)]
train=train[train['accuracy']<1000]

placeCounts=train['place_id'].value_counts()



valid=_train_[int(len(_train_)*trainRatio):]

/home/zhenjie/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [12]:
def featureFactory(dataFrame):
    dataFrame.loc[:,'xTimeY']=dataFrame.loc[:, 'x']*dataFrame.loc[:, 'y']
    dataFrame.loc[:,'xDivY']=dataFrame.loc[:, 'x'].div(dataFrame.loc[:, 'y'])

    dataFrame.loc[:,'accuracy']=np.log10(dataFrame.loc[:, 'accuracy'])
    
    dataFrame.loc[:,'day']=dataFrame.loc[:, 'time'].div(1440).map(int)
    dataFrame.loc[:,'weekday']=dataFrame.loc[:,'day']%7
    dataFrame.loc[:,'weekdayShifted']=(dataFrame.loc[:,'weekday']+3)%7
    dataFrame.loc[:,'dayInMonth']=dataFrame.loc[:, 'day']%(30)
    dataFrame.loc[:,'year']=dataFrame.loc[:,'day'].div(365).map(int)
    
    dataFrame.loc[:,'month']=(dataFrame.loc[:,'day']%365).div(30).map(int)
    dataFrame.loc[:,'hour']=dataFrame.loc[:,'time'].div(60).map(int)
    dataFrame.loc[:, 'hourInDay']=dataFrame.loc[:,'hour']%24
    dataFrame.loc[:, 'hourInDayShifted1']=(dataFrame.loc[:,'hourInDay']+6)%24
    dataFrame.loc[:, 'hourInDayShifted2']=(dataFrame.loc[:,'hourInDay']+12)%24
    dataFrame.loc[:, 'hourInDayShifted3']=(dataFrame.loc[:,'hourInDay']+18)%24

 
    dataFrame.loc[:, 'originalIndex']=xrange(len(dataFrame))

In [13]:
featureFactory(train)
featureFactory(valid)

In [5]:
test=pd.read_csv('../input/test.csv')
featureFactory(test)

In [6]:
g=train.groupby('place_id')
places=pd.DataFrame()
places['xMean']=g['x'].mean()
places['xStd']=g['x'].std()

places['yMean']=g['y'].mean()
places['yStd']=g['y'].std()

places['hourInDayMean']=g['hourInDay'].mean()
places['hourInDayStd']=g['hourInDay'].std()

places['monthMean']=g['month'].mean()
places['monthStd']=g['month'].std()

places['dayMean']=g['day'].mean()
places['dayStd']=g['day'].std()

places.head()

sTime=time.time()

train=train[(abs(train['y']-places.loc[train['place_id'], 'yMean'].values)<5*places.loc[train['place_id'], 'yStd'].values)\
         & (abs(train['x']-places.loc[train['place_id'], 'xMean'].values)<5*places.loc[train['place_id'], 'xStd'].values)\
           
           ]

print time.time()-sTime

,xMean,xStd,yMean,yStd,hourInDayMean,hourInDayStd,monthMean,monthStd,dayMean,dayStd
place_id,,,,,,,,,,
1000015801,2.690521,0.167190,5.550822,0.014536,16.277778,5.544245,3.291667,2.995008,128.638889,103.437306
1000017288,7.331718,0.338637,4.346353,0.010564,12.734940,4.123355,4.915663,3.390103,187.385542,111.164439
1000025138,0.987195,0.043501,5.570373,0.010376,12.624130,3.614283,5.624130,3.779722,231.658933,111.817801
1000052096,2.849199,0.446559,5.833482,0.011319,9.787356,3.360055,4.525862,3.551782,183.886494,118.262177
1000063498,3.867737,1.084438,7.547868,0.022191,16.684211,2.688507,7.052632,3.487656,266.210526,93.450866


173.348020077


In [14]:
nXRegions=50
nYRegions=50
xMin=0
xMax=2
yMin=0
yMax=2
nXRegions=nXRegions*int(xMax-xMin)/10
nYRegions=nYRegions*int(yMax-yMin)/10
# train=train[(train['x']<xMax) & (train['x']>=xMin) & (train['y']<yMax) & (train['y']>=yMin)]
# valid=valid[(valid['x']<xMax) & (valid['x']>=xMin) & (valid['y']<yMax) & (valid['y']>=yMin)]

print xMin, xMax, yMin, yMax, nXRegions, nYRegions

0 2 0 2 10 10


In [8]:
# def regionalForestPred(pred, classes, regionalPoints, data):

    
#     prob=[sorted(zip(p, classes))[-3:][::-1] for p in pred]
    
    
#     regionalPred=[]
#     for i in xrange(len(data)):
#         sampleY=data['y'].iloc[i]
#         sampleX=data['x'].iloc[i]
#         sampleDay=data['day'].iloc[i]

#         filteredPlaces=filter(lambda x: \
#                               sampleY<=yMaxMap[x[1]]+yAllowance and sampleY>=yMinMap[x[1]]-yAllowance and \
#                               sampleX<=xMaxMap[x[1]] and sampleX>=xMinMap[x[1]] and \
#                               sampleDay<=dayMaxMap[x[1]] and sampleDay>=dayMinMap[x[1]], prob[i])
#         if filteredPlaces:
#             filteredPlaces=map(lambda x: [x[0]*regionalPoints, x[1]], filteredPlaces)
#         regionalPred.append(filteredPlaces)
#     return regionalPred

In [23]:
def train_test(modelSerialization=False, resultSerialization=False, model='rf', predictSet=valid, th=0, xMargin=0.05, yMargin=0.025):
    
    if modelSerialization or resultSerialization:
        base='./'
        folderName=model+'-'+time.strftime('%c')
        folderPath=base+folderName+'/'
        if not os.path.exists(folderPath):
            os.makedirs(folderPath)
            
    
#     f=['x','y','xTimeY','xDivY', 'accuracy', 'day', 'weekday', 'hourInDay','hourInDayShifted2', 'month', 'year']
    f=['x','y', 'accuracy', 'day','weekday', 'hourInDay','hourInDayShifted2', 'month', 'year']
        
    count=0
    accuracies=[]
    FBscores=[]
    
    startTime=time.time()
    trainingTimes=[]
    predictionTimes=[]
    
    testNum=0

    roundNum=5

    for xNum, yNum in ((xn, yn) for xn in xrange(nXRegions) for yn in xrange(nYRegions)):

        xStep=round(1.0*(xMax-xMin)/nXRegions,roundNum)
        xStart=round(xMin+xNum*xStep,roundNum)
        xEnd=round(xStart+xStep,roundNum)

        yStep=round(1.0*(yMax-yMin)/nYRegions,roundNum)
        yStart=round(yMin+yNum*yStep,roundNum)
        yEnd=round(yStart+yStep,roundNum)
        
#         regionalTrain=train[(train['x']<xEnd) & (train['x']>=xStart) & (train['y']<yEnd) & (train['y']>=yStart)]

        regionalTrain=train[(train['x']<xEnd+xMargin) & (train['x']>=xStart-xMargin) & (train['y']<yEnd+yMargin) & (train['y']>=yStart-yMargin)]
#         print xStart, xEnd, yStart, yEnd, len(regionalTrain)

        
            
#         regionalTrain['sampleWeight']=


        placesCounts=regionalTrain['place_id'].value_counts()
        regionalTrain=regionalTrain[(placeCounts[regionalTrain['place_id'].values]>th).values]
        
        tail=len(regionalTrain)/6
        latest=regionalTrain.sort_values(by=['time'])[:tail]
        regionalTrain=regionalTrain.append(latest)
        
        
        addition=regionalTrain[regionalTrain['hourInDay']<=4]
        addition['hourInDay']=addition['hourInDay']+24
        regionalTrain=regionalTrain.append(addition)
        
        addition=regionalTrain[regionalTrain['hourInDay']>=20]
        addition['hourInDay']=addition['hourInDay']-24
        regionalTrain=regionalTrain.append(addition)


        
        if len(regionalTrain):
#             regionalPredictSet=predictSet[(predictSet['x']<xEnd-margin) & (predictSet['x']>=xStart+margin) & (predictSet['y']<yEnd-margin) & (predictSet['y']>=yStart+margin)]
            regionalPredictSet=predictSet[(predictSet['x']<xEnd) & (predictSet['x']>=xStart) & (predictSet['y']<yEnd) & (predictSet['y']>=yStart)]

            testNum+=len(regionalPredictSet)

            if model=='knn':
                fWeights={'x': 500, 'y': 1000, 'accuracy':0, 'day': 0,'dayInMonth': 0, 'weekday': 3,\
                          'hourInDay': 4, 'hourInDayShifted2': 0, 'month': 2, 'year': 10}
                for ft in f:
                    regionalTrain[ft]=regionalTrain[ft].values*fWeights[ft]
                    regionalPredictSet[ft]=regionalPredictSet[ft].values*fWeights[ft]
                
            else:
                fMean=regionalTrain[f].mean()
                fStd=regionalTrain[f].std()

                for ft in f:       
                    regionalTrain[ft]=(regionalTrain[ft].values-fMean[ft])/(fStd[ft])
                    regionalPredictSet[ft]=(regionalPredictSet[ft].values-fMean[ft])/(fStd[ft])
#                 print regionalTrain['x'].describe()

      
            le=LabelEncoder()
            regionalTrain['le']=le.fit_transform(regionalTrain['place_id'].values)

#             s_w=1**(15-regionalTrain['day'].values/30.0)
            
            trainingStartTime=time.time()
            if model=='rf':
                s_w_rf=3**(-regionalTrain['day'].values)
#                 s_w_rf=1.65-regionalTrain['day'].values
    #             clf=RandomForestClassifier(n_jobs=-1, n_estimators=300, max_features=None).fit(regionalTrain[f], regionalTrain['le'])   
            
                clf=RandomForestClassifier(n_jobs=-1, n_estimators=150, random_state=0, min_samples_split=4).fit(regionalTrain[f], regionalTrain['le'], sample_weight=s_w_rf)  

            if model=='xgb':
                clf=XGBClassifier(learning_rate=0.02, n_estimators=220, objective='multi:softprob', max_depth=3, seed=0).fit(regionalTrain[f], regionalTrain['le'])
            
            
            if model=='knn':
                clf=KNeighborsClassifier(n_neighbors=25, weights='distance',metric='manhattan', n_jobs=-1).fit(regionalTrain[f], regionalTrain['le'])
         
            trainingTimes.append(time.time()-trainingStartTime)
            
            
            if modelSerialization:
                modelFileName='{:04d}-{}-{}-{}-{}.clf'.format(count, xStart, xEnd, yStart, yEnd)
                modelFilePath=folderPath+modelFileName
                with open(modelFilePath, 'ab+') as fo:
                    pickle.dump(clf, fo, pickle.HIGHEST_PROTOCOL)
            
            
            predictionStartTime=time.time()
            prob=clf.predict_proba(regionalPredictSet[f])
            predictionTimes.append(time.time()-predictionStartTime)
            
            pred=[sorted(zip(p, clf.classes_))[-3:][::-1] for p in prob]
            prediction=le.inverse_transform([zip(*p)[1] for p in pred])
            confidence=[zip(*p)[0] for p in pred]
            
            
                
            fbscoreForSerialization=-1
            if 'place_id' in predictSet.columns:
                regionalAccuracy=1.0*sum([regionalPredictSet['place_id'].iloc[i] in prediction[i] for i in xrange(len(regionalPredictSet))])/len(regionalPredictSet)
                regionalConfidence=[sum(z)/len(regionalPredictSet) for z in zip(*confidence)]
                regionalConfidence3=sum(regionalConfidence)

                fbAccuracy=0
                fbAccuracy+=1.0*sum([regionalPredictSet['place_id'].iloc[i] in prediction[i][:1] for i in xrange(len(regionalPredictSet))])/len(regionalPredictSet)
                fbAccuracy+=1.0/2*sum([regionalPredictSet['place_id'].iloc[i] in prediction[i][1:2] for i in xrange(len(regionalPredictSet))])/len(regionalPredictSet)
                fbAccuracy+=1.0/3*sum([regionalPredictSet['place_id'].iloc[i] in prediction[i][2:3] for i in xrange(len(regionalPredictSet))])/len(regionalPredictSet)
                FBscores.append(fbAccuracy)
                print 'region {}: {},{} accuracy: {},  fbAccu: {}, confidence: {}:'.format(count, xNum, yNum, regionalAccuracy, fbAccuracy, regionalConfidence3)
                accuracies.append(regionalAccuracy)
                if resultSerialization:
                    fbscoreForSerialization=fbAccuracy
                
            if resultSerialization:
                resultFileName='{:04d}-{}-{}-{}-{}.rst'.format(count, xStart, xEnd, yStart, yEnd)
                resultFilePath=folderPath+resultFileName
                predColumns=zip(*prediction)
                confColumns=zip(*confidence)

                
                if len(predColumns[2])!=len(prediction):
                    print 'missing values',count, len(predColumns[0])
                
                results=pd.DataFrame({'originalIndex': regionalPredictSet['originalIndex'].tolist(),\
                                      'x':regionalPredictSet['x'].tolist(), 'y':regionalPredictSet['y'].tolist(), \
                                      'accuracy':regionalPredictSet['accuracy'].tolist(), 'pred0':predColumns[0], \
                                      'pred1':predColumns[1], 'pred2':predColumns[2], 'conf0': confColumns[0], \
                                      'conf1': confColumns[1], 'conf2': confColumns[2], 'regionalFBScore': [fbscoreForSerialization]*len(regionalPredictSet)})

                results.to_csv(resultFilePath)



        count+=1
        if count%10==0:

            print '{} : total time {} s.'.format(count, time.time()-startTime)
            print 'Average training Time: ', sum(trainingTimes)/len(trainingTimes)
            print 'Average prediction Time: ', sum(predictionTimes)/len(predictionTimes)
            print 'Average FB Score', np.mean(FBscores)
            print 
            
            startTime=time.time()
            trainingTimes=[]
            predictionTimes=[]


    print 
    print np.mean(accuracies)
    print np.var(accuracies)
    print
    print np.mean(FBscores)
    print testNum
    print 'done'
    
    
    
    

In [ ]:
train_test(modelSerialization=False, model='xgb', resultSerialization=False, predictSet=valid, th=3, xMargin=0.06, yMargin=0.03)

In [14]:
def makeSubmission(resultFolder):
    results=[]
    for f in os.listdir(resultFolder):
        if f.endswith('.rst'):
            fi=resultFolder+'/'+f
            results.append(pd.read_csv(fi))
    results=pd.concat(results)
        
    results.sort(['originalIndex'], inplace=True)
    return results

In [ ]:
results=makeSubmission('../submissions/xgb-0.03-50Trees/')
predictions=results[['pred0','pred1','pred2']]
submit=pd.DataFrame()
# submit.loc[:,'row_id']=np.arange(len(predictions))
submit.loc[:,'place_id']=predictions[['pred0', 'pred1', 'pred2']].apply(lambda x: ' '.join([str(nx) for nx in x]), axis=1)
submit.loc[:,'row_id']=np.arange(len(predictions))


In [ ]:
submit=submit[['row_id','place_id' ]]
submit.set_index('row_id', inplace=True)
submit.to_csv('../submissions/xgb0.03_50Trees.csv')


In [ ]:
p300=pd.read_csv('../submissions/rf300Trees10_250.csv').head()
p50=pd.read_csv('../submissions/rf50Trees10_250.csv').head()
p300
p50

In [ ]:
others=pd.read_csv('../submissions/rf_submission_2016-06-03-19-57.csv')

In [ ]:
results=makeSubmission('rf-Tue Jun 14 18:18:14 2016')
predictions=results[['pred0','pred1','pred2']].values

realXMin=0
realXMax=10.1
realYMin=0
realYMax=10.1

real=_train_[int(len(_train_)*trainRatio):]

real=real[(real['x']<realXMax) & (real['x']>=realXMin) & (real['y']<realYMax) & (real['y']>=realYMin)]
real=real[['place_id']]

print len(real)
print len(predictions)

# print len(predictions), len(real), results['originalIndex'].value_counts()

totalHit=sum([real['place_id'].iloc[i] in predictions[i] for i in xrange(len(predictions))])
score=0
score+=1.0*sum([real['place_id'].iloc[i] in predictions[i][:1] for i in xrange(len(predictions))])
score+=1.0/2*sum([real['place_id'].iloc[i] in predictions[i][1:2] for i in xrange(len(predictions))])
score+=1.0/3*sum([real['place_id'].iloc[i] in predictions[i][2:3] for i in xrange(len(predictions))])

print 1.0*totalHit/len(predictions)
print score/len(predictions)



In [ ]:
real.head()

results[results['originalIndex']==934546]

In [ ]:
r=pd.read_csv('rf-Sun Jun 12 12:17:04 2016/0000-0.0-1.0-0.0-0.04.rst')

In [ ]:
r.head()